In [1]:
using LinearAlgebra, Distributions, CSV, DataFrames, ProgressMeter
include("../../slv.jl")
using .slv
include("../../equation.jl")
using .equation

include("../../LSMC/lsmc_bsde.jl")

In [2]:
S0 = 95.0; v0 = 0.4; ρ = 0.8; T = 1.0; b = 0.04; β = 0.25; σ₀ = 0.25; η = 0.5; α = 0.3;
R = 0.07; r = 0.006; K1 = 95.0; K2 = 105.0;
slvm = HypHypSLV(S0::Float64, v0::Float64, ρ::Float64, b::Float64, β::Float64, σ₀::Float64, η::Float64, α::Float64);
Π = [1.0 ρ; ρ 1.0]; cholΠ=cholesky(Π).L

drift(x) = [slvm.ω(abs(x[1]),abs(x[2])), slvm.μ(abs(x[2]))]
diffusion(x) = Diagonal(
        [slvm.m(abs(x[2]))*slvm.Γ(abs(x[1])), slvm.σ(abs(x[2]))]
        )*cholΠ
driver(t, x, y, z) = (
    -r*max(y-z'*inv(diffusion(x))*x, 0.0)
    -R*min(y-z'*inv(diffusion(x))*x, 0.0)
    -z'*inv(diffusion(x))*drift(x)
)
terminal(x) = max(x[1]-K1,0.0)-2.0*max(x[1]-K2,0.0)

terminal (generic function with 1 method)

In [3]:
#### LSMC
#### Laguerre polynomials
function Laguerre1D(x,k)
    if k == 0
        return 1.0
    elseif k == 1
        return (1.0-x)
    elseif k == 2
        return (x^2-4*x+2)/2
    elseif k == 3
        return (-x^3+9*x^2-18*x+6)/6
    elseif k == 4
        return (x^4-16*x^3+72*x^2-96*x+24)/24
    elseif k == 5
        return (-x^5+25*x^4-200*x^3+600*x^2-600*x+120)/120
    elseif k == 6
        return (x^6-36*x^5+450*x^4-2400*x^3+5400*x^2-4320*x+720)/720
    elseif k == 7
        return (-x^7+49*x^6-882*x^5+7350*x^4-29400*x^3+52920*x^2-35280*x+5040)/5040
    elseif k == 8
        return (x^8-64*x^7+1568*x^6-18816*x^5+117600*x^4-376320*x^3+564480*x^2-322560*x+40320)/40320
    elseif k == 9
        return (-x^9+81*x^8-2592*x^7+42336*x^6-381024*x^5+1905120*x^4-5080320*x^3+6531840*x^2-3265920*x+362880)/362880
    else
        return 0.0
    end
end

function bases2D(x, poly)
    vec = zeros((poly+1)^2)
    for k = 0:poly
        for l = 0:poly
            vec[(poly-1)*k+l+1] = Laguerre1D(x[1], k)*Laguerre1D(x[2], l)
        end
    end
    return vec
end

M=2^20

1048576

In [3]:
N=6
poly = N
ba = (x->bases2D(x, poly))
samples = 50;
@showprogress for s0 in [95.0,100.0,105]    
    bsde = BSDE(T, [s0, v0], drift, diffusion, driver, terminal);
#### Generate 50 independent samples
    Y0 = zeros(samples);
    exc_start = time()
    @showprogress for k in 1:samples
        Y0[k] = LSMC(bsde, ba, M, N);
    end
    exc_stop = time()
    elapsed_time = exc_stop-exc_start
    print(string(":LSMCY_0(",s0,", ",v0,")=",mean(Y0),"/std=",std(Y0),". (",elapsed_time," seconds)\n"))
    print("\n")
    flush(stdout)
end

Progress: 100%|█████████████████████████████████████████| Time: 1:00:41


:LSMCY_0(95.0, 0.4)=4.466563705718711/std=0.013817023952866922. (3641.4754469394684 seconds)



Progress: 100%|█████████████████████████████████████████| Time: 0:59:55


:LSMCY_0(100.0, 0.4)=5.795169096561708/std=0.019986846917120583. (3595.6368680000305 seconds)



Progress: 100%|█████████████████████████████████████████| Time: 1:00:05


:LSMCY_0(105.0, 0.4)=5.661857959343783/std=0.02496535687289554. (3605.5047318935394 seconds)



Progress: 100%|█████████████████████████████████████████| Time: 3:00:43


In [7]:
N=7
poly = N
ba = (x->bases2D(x, poly))
samples = 50;
@showprogress for s0 in [95.0,100.0,105]    
    bsde = BSDE(T, [s0, v0], drift, diffusion, driver, terminal);
#### Generate 50 independent samples
    Y0 = zeros(samples);
    exc_start = time()
    @showprogress for k in 1:samples
        Y0[k] = LSMC(bsde, ba, M, N);
    end
    exc_stop = time()
    elapsed_time = exc_stop-exc_start
    print(string(":LSMCY_0(",s0,", ",v0,")=",mean(Y0),"/std=",std(Y0),". (",elapsed_time," seconds)\n"))
    print("\n")
    flush(stdout)
end

Progress: 100%|█████████████████████████████████████████| Time: 1:25:16


:LSMCY_0(95.0, 0.4)=4.450400046372636/std=0.014021001756184844. (5116.942748069763 seconds)



Progress: 100%|█████████████████████████████████████████| Time: 1:25:41


:LSMCY_0(100.0, 0.4)=5.7897232021462495/std=0.020746976449467963. (5141.017956972122 seconds)



Progress: 100%|█████████████████████████████████████████| Time: 1:25:11


:LSMCY_0(105.0, 0.4)=5.63837802054421/std=0.02985267408385575. (5111.5640869140625 seconds)



Progress: 100%|█████████████████████████████████████████| Time: 4:16:09


In [4]:
N=8
poly = N
ba = (x->bases2D(x, poly))
samples = 50;
@showprogress for s0 in [95.0,100.0,105]    
    bsde = BSDE(T, [s0, v0], drift, diffusion, driver, terminal);
#### Generate 50 independent samples
    Y0 = zeros(samples);
    exc_start = time()
    @showprogress for k in 1:samples
        Y0[k] = LSMC(bsde, ba, M, N);
    end
    exc_stop = time()
    elapsed_time = exc_stop-exc_start
    print(string(":LSMCY_0(",s0,", ",v0,")=",mean(Y0),"/std=",std(Y0),". (",elapsed_time," seconds)\n"))
    print("\n")
    flush(stdout)
end

Progress: 100%|█████████████████████████████████████████| Time: 2:03:16


:LSMCY_0(95.0, 0.4)=4.444755567038165/std=0.014545782052124397. (7396.037760972977 seconds)



Progress: 100%|█████████████████████████████████████████| Time: 2:02:54


:LSMCY_0(100.0, 0.4)=5.785148743797801/std=0.01955139550258841. (7374.197238922119 seconds)



Progress: 100%|█████████████████████████████████████████| Time: 2:01:58


:LSMCY_0(105.0, 0.4)=5.629258283162759/std=0.03495093357242778. (7318.980121135712 seconds)



Progress: 100%|█████████████████████████████████████████| Time: 6:08:09


In [5]:
N=9
poly = N
ba = (x->bases2D(x, poly))
samples = 50;
@showprogress for s0 in [95.0,100.0,105]    
    bsde = BSDE(T, [s0, v0], drift, diffusion, driver, terminal);
#### Generate 50 independent samples
    Y0 = zeros(samples);
    exc_start = time()
    @showprogress for k in 1:samples
        Y0[k] = LSMC(bsde, ba, M, N);
    end
    exc_stop = time()
    elapsed_time = exc_stop-exc_start
    print(string(":LSMCY_0(",s0,", ",v0,")=",mean(Y0),"/std=",std(Y0),". (",elapsed_time," seconds)\n"))
    print("\n")
    flush(stdout)
end

Progress: 100%|█████████████████████████████████████████| Time: 2:55:05


:LSMCY_0(95.0, 0.4)=4.4393352198291165/std=0.018057431997380698. (10505.512458086014 seconds)



Progress: 100%|█████████████████████████████████████████| Time: 2:54:35


:LSMCY_0(100.0, 0.4)=5.786427560251386/std=0.023520877777286647. (10475.728768110275 seconds)



Progress: 100%|█████████████████████████████████████████| Time: 2:53:51


:LSMCY_0(105.0, 0.4)=5.635450764821429/std=0.030646236083357658. (10431.613009929657 seconds)



Progress: 100%|█████████████████████████████████████████| Time: 8:43:32
